### Reproduce m_plus_wd

#### Purpose

To reproduce the MS+HeWD runs in the m_plus_wd project using tools from qol.

In [4]:
from qol.mesa.launcher import *
import qol.paths as paths

In [3]:
from qol.mesa.launcher import *
import qol.paths as paths

# Goal: reproduce something similar to m_plus_wd
net_name = 'pp_cno_extras_o18_ne22.net'
pgstar_path = f'{paths.qol_path}/mesa/resources/r24.08.1/inlist_pgstar'
Mcore_in_Msun = 0.15 # core mass, free parameter
Menv_in_Msun = 0.5 # envelope mass, free parameter
T_WD = 8000 # WD temperature

# Step 0: working directory
work = MesaWorkingDirectory(run_path='/Users/nrui/Desktop/m_plus_wd_sequel/')
work.copy_history_columns_list(f'{paths.qol_path}/mesa/resources/r24.08.1/history_columns.list')
work.copy_profile_columns_list(f'{paths.qol_path}/mesa/resources/r24.08.1/profile_columns.list')

# Step 1: make RG, evolve to desired core mass
inlist = MesaInlist('inlist_evolve_rg', LOGS_dir='LOGS_evolve_rg/')
inlist.show_pgstar(pgstar_path)

inlist.initial_mass(1.2)
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

inlist.energy_eqn_option('eps_grav')

inlist.he_core_boundary_h1_fraction(1e-3)
inlist.he_core_mass_limit(Mcore_in_Msun)
inlist.save_final_model('rg.mod')

work.add_task(inlist)

# Step 2: remove mass from RG
inlist = MesaInlist('inlist_strip_rg', LOGS_dir='LOGS_strip_rg/')
inlist.show_pgstar(pgstar_path)

inlist.load_model('rg.mod')
inlist.relax_initial_mass(M_new_Msun=Mcore_in_Msun, lg_max_abs_mdot=-3)
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

inlist.energy_eqn_option('eps_grav')

inlist.max_model_number(1)
inlist.save_final_model('hot_he_wd.mod')

work.add_task(inlist)

# Step 3: cool WD to desired temperature
inlist = MesaInlist('inlist_cool_wd', LOGS_dir='LOGS_cool_wd/')
inlist.show_pgstar(pgstar_path)

inlist.load_model('hot_he_wd.mod')
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

inlist.energy_eqn_option('eps_grav')

inlist.Teff_lower_limit(T_WD)
inlist.save_final_model('cool_he_wd.mod')

work.add_task(inlist)

# Step 4: create envelope matching core model
script = MesaPythonScript('inner_bc.py',
            template=f'{paths.qol_path}/mesa/templates/scripts/call_create_env_inlist_from_core.py',
              const_args=[Menv_in_Msun], prereqs=['cool_he_wd.mod'], products=['inlist_env_inner_bc'])

work.add_task(script)

# Step 5: run envelope model to th eq (no dxdt_nuc)
inlist = MesaInlist('inlist_env_to_th_eq', LOGS_dir='LOGS_env_to_th_eq/')
inlist.show_pgstar(pgstar_path)

inlist.read_extra_inlist(namelist='star_job', rel_path='inlist_env_inner_bc', category='relax inner BC to accommodate core model')

inlist.create_pre_main_sequence_model(True)
inlist.initial_mass(Menv_in_Msun)
inlist.initial_y(0.28)
inlist.initial_z(0.02)
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

inlist.min_timestep_limit(1e-12)
inlist.energy_eqn_option('dedt')
inlist.use_gold_tolerances(False)
inlist.convergence_ignore_equL_residuals(True)
inlist.set_max_num_retries(3000)
inlist.limit_for_rel_error_in_energy_conservation(-1.)
inlist.disable_dxdt_from_nuclear_burning()

inlist.max_age(1e10)
inlist.save_final_model('env_th_eq.mod')

work.add_task(inlist)

# Step 6: stitch core and envelope together
script = MesaPythonScript(rel_path='merge.py',
            template=f'{paths.qol_path}/mesa/templates/scripts/call_create_shell_burning_remnant.py',
            prereqs=['cool_he_wd.mod', 'env_th_eq.mod'],
            products=['remnant_init.mod'])

work.add_task(script)

# Step 7: run remnant into HSE (1e5 yr, no dxdt_nuc or mix)
inlist = MesaInlist('inlist_remnant_ringdown', LOGS_dir='LOGS_remnant_ringdown/')
inlist.show_pgstar(pgstar_path)

inlist.load_model('remnant_init.mod')
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

inlist.enable_hydrodynamics()
inlist.add_drag_for_HSE(drag_coefficient=1.)
inlist.energy_eqn_option('eps_grav')
inlist.min_timestep_limit(1e-12)
inlist.use_gold_tolerances(False)
inlist.convergence_ignore_equL_residuals(True)
inlist.disable_dxdt_from_nuclear_burning()
inlist.disable_mixing()

inlist.max_age(1e5)
inlist.save_final_model('remnant_hse.mod')

work.add_task(inlist)

# Step 8: run remnant with full physics for as long as possible (max_age = 1e10 yr)
inlist = MesaInlist('inlist_evolve_remnant', LOGS_dir='LOGS_evolve_remnant/')
inlist.show_pgstar(pgstar_path)

# inlist.enable_hydrodynamics()

inlist.load_model('remnant_hse.mod')
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

inlist.energy_eqn_option('eps_grav')

inlist.max_age(1e10)
inlist.save_final_model('remnant_final.mod')

work.add_task(inlist)

# add additional steps:
# - at flash, terminate
# - run through flash with some convergence controls relaxed, end flash at zacheb
# - continue through clump and agb with winds

# - maybe expand net to have even more alpha elements in it

work.save_directory()

<function qol.mesa.read.read_data(fname, remove_duplicates=True)>